# CRAWL DỮ LIỆU SOUNDCLOUD VỚI API

## BẢNG PHÂN CÔNG CÔNG VIỆC

STT&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;Thông tin sinh viên&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;Mức độ hoàn thành&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;Công việc
<br>
1&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&ensp;&ensp;&nbsp;Nguyễn Đăng Thi-19127276&emsp;&emsp;&emsp;&emsp;&emsp;&nbsp;&nbsp;&nbsp;100%&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&nbsp;&nbsp;&nbsp;&ensp;Crawl Api với track và playlist
<br>
2&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&nbsp;Lê Minh Trí-19127592&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&ensp;100%&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&nbsp;&nbsp;&nbsp;&ensp;Crawl data bằng Selenium với track và playlist
<br>
3&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&nbsp;Phan Vĩ Giai-19127639&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;100%&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&nbsp;&nbsp;&nbsp;&ensp;Crawl Api với users
<br>
4&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&nbsp;Võ Nhất Huy-19127642&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&ensp;&nbsp;&nbsp;100%&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&nbsp;&nbsp;&nbsp;&ensp;Crawl data bằng Selenium với user và link lấy link dẫn từ home page tới playlist

## CÁCH CHẠY PROJECT
**Chọn Kernel -> Restart & Run All**

**Task**

1.   &emsp;Parse HTML
2.   &emsp;Get API
3.   &emsp;Write report

--- 

**Data**

1.   User
2.   Playlist
3.   Track

***File structure will construct like below***
<br>
User
<br>
id&emsp;&emsp;id_playlist
<br>
1&emsp;&emsp;&ensp;1
<br>
1&emsp;&emsp;&ensp;2
<br>

Playlist
<br>
id&emsp;&emsp;tracks
<br>
1&emsp;&emsp;&ensp;357,381,1
<br>
2&emsp;&emsp;&ensp;357,381,1
<br>

Track
<br>
id&emsp;&emsp;&emsp;name
<br>
1&emsp;&emsp;&emsp;A
<br>
357&emsp;&emsp;B
<br>
381&emsp;&emsp;C

***
# Thu thập data từ Soundcloud bằng cách parse HTML - sử dụng Selenium
***

### Các thư viện được sử dụng

In [1]:
import os
import io
import csv
import time
import requests
import requests_cache
import pandas as pd
from PIL import Image
from threading import Thread, Barrier
import selenium
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait  # for implicit and explict waits
from selenium.webdriver.chrome.options import Options  # for suppressing the browser
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.common.by import By

Để sử dụng selenium, ta cần tạo một webdrive để điều khiển trình duyệt, ở đây trình duyệt được sử dụng là Chrome. Đoạn code này sẽ làm 1 số việc

-    Check browser version hiện có của máy
-    Check version của driver, nếu không biết thì nó sẽ dùng version mới nhất
-    Tạo ra biến môi trường cho Webdriver, phục vụ cho selenium.

In [18]:
s=Service(ChromeDriverManager().install())
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument('disable-infobars')
chrome_options.add_argument(
            '--disable-gpu') if os.name == 'nt' else None  # Windows workaround
chrome_options.add_argument("--verbose")
chrome_options.add_argument("--no-default-browser-check")
chrome_options.add_argument("--ignore-ssl-errors")
chrome_options.add_argument("--allow-running-insecure-content")
chrome_options.add_argument("--disable-web-security")

chrome_options.add_argument(
            "--disable-feature=IsolateOrigins,site-per-process")
chrome_options.add_argument("--no-first-run")
chrome_options.add_argument("--disable-notifications")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-translate")
chrome_options.add_argument("--ignore-certificate-error-spki-list")
chrome_options.add_argument(
            "--disable-blink-features=AutomationControllered")
chrome_options.add_experimental_option('useAutomationExtension', False)
prefs = {"profile.default_content_setting_values.notifications": 2}
chrome_options.add_experimental_option("prefs", prefs)
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_experimental_option(
            "excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option(
            "prefs", {"profile.managed_default_content_settings.images": 2})
chrome_options.add_argument('disable-infobars')
chrome_options.add_argument('--blink-settings=imagesEnabled=false')


driver = webdriver.Chrome(service=s, options = chrome_options)



====== WebDriver manager ======
Current google-chrome version is 95.0.4638
Get LATEST chromedriver version for 95.0.4638 google-chrome
Driver [/home/waterting/.wdm/drivers/chromedriver/linux64/95.0.4638.69/chromedriver] found in cache


Ngoài ra với option 'headless', webdrive có thể điều khiển mà không cần phải mở browser

### Scroll
Chức năng scroll được sử dụng để load đến cuối một trang web, điều này đảm bảo sẽ lấy được nhiều thông tin nhất có thể từ trang web, vì nếu không thực hiện scroll, một số phần ở dưới có thể không được load trong HTML khiến việc lấy dữ liệu không đạt hiệu quả cao.


In [4]:
def scroll_to_bottom(driver):

    old_position = 0
    new_position = None

    while new_position != old_position:
        # Get old scroll position
        old_position = driver.execute_script(
                ("return (window.pageYOffset !== undefined) ?"
                 " window.pageYOffset : (document.documentElement ||"
                 " document.body.parentNode || document.body);"))
        # Sleep and Scroll
        time.sleep(1)
        driver.execute_script((
                "var scrollingElement = (document.scrollingElement ||"
                " document.body);scrollingElement.scrollTop ="
                " scrollingElement.scrollHeight;"))
        # Get new position
        new_position = driver.execute_script(
                ("return (window.pageYOffset !== undefined) ?"
                 " window.pageYOffset : (document.documentElement ||"
                 " document.body.parentNode || document.body);"))

***
### Parse HTML

Với soundcloud, chúng ta không được cung cấp một nguồn thông tin đầy đủ về user, track hay playlist. Do đó để khai thác được các thông tin theo yêu cầu, ta cần duyệt từ trang chủ.
Tại trang chủ 'https://soundcloud.com/discover', ta có thể thấy có rất nhiều playlists (được tạo bởi soundcloud hoặc người dùng), từ những playlists này dễ dàng tìm được thông tin về user đăng tải những bài hát đó. Với số lượng không nhỏ các playlists, số lượng user có thể tìm được từ đó tất nhiên cũng dễ dàng thỏa mãn yêu cầu đề bài.

In [5]:
def get_all_gallery_in_home(driver):
    home = 'https://soundcloud.com/discover'
    driver.get(home)
    time.sleep(1)
    #scroll till end of page
    scroll_to_bottom(driver)
    #execute all slide button 
    find_result = driver.find_elements(By.XPATH, "//div[@class = 'playableTile__artwork']/a")
    linkResults = []
    for each_res in find_result:
        if 'sets' in  str(each_res.get_attribute('href')):
            linkResults.append(each_res.get_attribute('href'))
    return linkResults

link_result = get_all_playlist_in_home()
print(len(link_result))

In [6]:
def get_user_link(gallery_link, driver): #get tag <a href="/touliver" class="trackItem__username sc-link-light sc-link-secondary">Touliver</a>
    driver.get(gallery_link)
    time.sleep(1)
    scroll_to_bottom(driver)
    find_result = driver.find_elements(By.XPATH, "//a[@class = 'trackItem__username sc-link-light sc-link-secondary']")
    linkResults = []
    for each_res in find_result:
        linkResults.append(each_res.get_attribute('href'))
    return linkResults

#### Thu thập data của user
User được lấy từ các playlists trong trang chủ sẽ được lưu ở dạng link dẫn tới trang của user đó. Với các đường link này ta có thể khai thác được các thông tin của user, cũng như các playlist mà user đó đã tạo(nếu có).

Hàm crawl_data_from_user_link cho phép chúng ta dễ dàng lấy được từ trang của user bao gồm: username, số lượng người theo dõi, số người đang theo dõi, số lượng track, số lượng playlist, ...

Cũng trong hàm này chúng ta cũng lấy được được link dẫn tới các playlists, nếu user không tạo playlist nào thì kết quả trả ra là 'None'

Thông tin của user và các đường dẫn đến playlist được đưa vào mảng để return, việc này giúp cho truy cập vào dữ liệu từ bên ngoài dễ dàng hơn khi chỉ cần truy cập vào các phần tử trong mảng. Với phần tử [0] là thông tin của user, [1] là mảng các đường dẫn tới playlist.

In [7]:
def crawl_data_from_user_link(user_link, driver):
    playlists_link = user_link + '/sets'
    #check if class="sc-link-primary soundTitle__title sc-link-dark sc-text-h4" is exist / or <div class="emptyNetworkPage__image"></div> is not exist
    #if exist, get link to play list
    #else playlist = none
    driver.get(playlists_link)
    time.sleep(1)
    scroll_to_bottom(driver)
    user_name = driver.find_elements(By.XPATH, "//div[@class = 'profileHeaderInfo__content sc-media-content']/h2")
    name = driver.find_elements(By.XPATH, "//h3[@class = 'profileHeaderInfo__additional g-type-shrinkwrap-block theme-dark g-type-shrinkwrap-large-secondary']")
    if len(name)!=0:
        name = name[0].text
    else:
        name = 'None'
    location = driver.find_elements(By.XPATH, "//h3[@class = 'profileHeaderInfo__additional g-type-shrinkwrap-block theme-dark g-type-shrinkwrap-large-secondary sc-mt-1x']")
    if len(location) != 0:
        location = location[0].text
    else:
        location = 'None'
    followers = driver.find_elements(By.XPATH, "//div[@class = 'infoStats__value sc-font-light']")[0].get_attribute('innerHTML')
    following = driver.find_elements(By.XPATH, "//div[@class = 'infoStats__value sc-font-light']")[1].get_attribute('innerHTML')
    tracks = driver.find_elements(By.XPATH, "//div[@class = 'infoStats__value sc-font-light']")[2].get_attribute('innerHTML')
    playlist_count = 0
    if driver.find_elements(By.XPATH, "//li[contains(@class, 'soundList__item')]") == []:   #playlists is empty
        return [[user_name[0].text, name, location, tracks, playlist_count, followers, following], 'None']
    else:   #get all link to playlist in playlists tab
        find_result = driver.find_elements(By.XPATH, "//a[@class = 'sc-link-primary soundTitle__title sc-link-dark sc-text-h4']")
        playlists_link = []
        for each_res in find_result:
            if each_res.get_attribute('href')!= None and "sets/" in each_res.get_attribute('href'):
                playlist_count +=1
                playlists_link.append(each_res.get_attribute('href'))
        return [[user_name[0].text, name, location, tracks, playlist_count, followers, following], playlists_link]

    


#### Test hàm crawl_data_from_user_link

In [ ]:
print(crawl_data_from_user_link('https://soundcloud.com/ph-ph-i-s')[0], driver)

#### Thu thập data của track
Lý do mà chúng ta cần quan tâm tới việc thu thập dữ liệu của track trước khi thu thập dữ liệu của playlist là vì ta cần ID của track để lưu vào playlist. 
Cũng tương tự như user, dữ liệu được trả về ở dạng mảng để dễ truy cập. 
Output của hàm crawl_track_data bao gồm:
- track_name: Tên bài nhạc
- in_playlist: Bài nhạc thuộc playlist nào
- Owner: Người sở hữu
- release_date: Ngày bài nhạc được thêm vào playlist
- duration: Thời lượng
- play: số người đã nghe
- like: số lượt thích
- repost: số lượt đăng lại bài nhạc
- number_comment: số lượt comment

In [8]:
def crawl_track_data(link_track, driver):
    driver.get(link_track)
    time.sleep(1)
    scroll_to_bottom(driver)

    track_name = driver.find_elements(By.XPATH,"//div[@class = 'soundTitle__titleHeroContainer']/h1/span")[0].text
    owner = driver.find_elements(By.XPATH,"//div[@class = 'soundTitle__usernameHeroContainer']/h2/a")[0].text
    release_date = driver.find_elements(By.XPATH, "//time[@class = 'relativeTime']")[0].get_attribute('title')
    duration = driver.find_elements(By.XPATH, "//div[@class = 'playbackTimeline__duration']")[0].text
    duration = duration[duration.rfind('\n')+1:]
    tmp = driver.find_elements(By.XPATH, "//li[@class = 'sc-ministats-item']")
    play = tmp[0].text
    play = play[play.rfind('\n')+1:]
    like = tmp[1].text
    like = like[like.rfind('\n')+1:]
    repost = tmp[2].text
    repost = repost[repost.rfind('\n')+1:]
    number_comment = driver.find_elements(By.XPATH, "//span[@class = 'commentsList__actualTitle']")
    if len(number_comment) == 0:
        number_comment = 0
    else: 
        number_comment = driver.find_elements(By.XPATH, "//span[@class = 'commentsList__actualTitle']")[0].text
        number_comment = number_comment[:number_comment.rfind(" ")]
    return [track_name, owner, release_date, duration, play, like, repost, number_comment]

#### Test thu thập dữ liệu một bài nhạc

In [91]:
crawl_track_data('https://soundcloud.com/lilcandypaint/selfishprod5heriff?in=lilcandypaint/sets/its-still-purple', driver)

['Selfish(prod.menohbeats)',
 'lilcandypaint *Candypaint',
 '19 February 2020',
 '2:00',
 '184K',
 '3,594',
 '125',
 '85']

#### Thu thập dữ liệu playlist
Hàm getInfoPlaylist thu thập dữ liệu từ các playlists của người dùng(nếu có). Input của hàm gồm link dẫn tới playlist và dataframe dùng để lưu dữ liệu các track trong playlist đó. 
Khi thực hiện hàm getInfoPlaylist, hàm lấy dữ liệu trong trang web playlist, bao gồm cả các đường link dẫn tới các tracks trong playlist đó. 

Link dẫn tới các playlist được lưu vào một mảng list_track. Khi thu thập dữ liệu track sẽ duyệt qua các đường link trong mảng, sau đó lưu vào dataframe

Output của hàm này bao gồm:
- playlist_name: Tên playlist
- playlist_owner: Người tạo playlist
- release_date: Ngày tạo playlist
- number_track: số track có trong playlist
- total_time: Tổng thời gian của các track trong playlist
- trackIds: ID của các tracks, dữ liệu này được từng track trả về, ghép với nhau ở dạng string, ngăn cách bởi dấu ","
- like: luợt like playlist
- repost: lượt repost playlist

In [9]:
def getInfoPlaylist(link, df_track, driver):
    driver.get(link)
    time.sleep(1)
    scroll_to_bottom(driver)
    playlist_name = driver.find_elements(By.XPATH, "//div[@class = 'soundTitle__titleHeroContainer']/h1/span")[0].text
    playlist_owner = driver.find_elements(By.XPATH, "//div[@class = 'soundTitle__usernameHeroContainer']/h2/a")[0].text
    release_date = driver.find_elements(By.XPATH, "//time[@class = 'relativeTime']")[0].get_attribute('title')
    track_count = driver.find_elements(By.XPATH, "//div[@class = 'playlistTrackCount sc-font']/div/div")
    number_track = track_count[0].get_attribute('innerHTML')
    total_time = track_count[3].get_attribute('innerHTML')
    playlist_stat = driver.find_elements(By.XPATH, "//div[@class = 'listenEngagement__footer']/ul/li")
    like = 0
    repost = 0
    if len(playlist_stat) == 2:
        like = playlist_stat[0].get_attribute('title')
        like = like[:like.rfind(" ")]
        repost = playlist_stat[1].get_attribute('title')
        repost = repost[:repost.rfind(" ")]
    elif len(playlist_stat) == 1:
        playlist_stat = playlist_stat[0].get_attribute('title')
        if 'like' in playlist_stat:
            like = playlist_stat[:playlist_stat.rfind(" ")]
        else:
            repost = playlist_stat[:playlist_stat.rfind(" ")]
    trackIds = "None"
    return [playlist_name, playlist_owner, release_date, number_track, total_time, trackIds, like, repost]


***
## Thu thập dữ liệu các track trong một playlist
Đối với các track trong playlist, để thu thập được nhiều trường nhất ta cần phải truy cập vào từng link để parse HTML. 

Tuy nhiên hệ quả của việc này là sẽ rất tốn thời gian nếu như thực hiện chạy cùng lúc với quá trình thu thập dữ liệu của một playlist nào đó.
Do vạy thao tác này cần được tách riêng ra và thực hiện sau cùng, khi mà user và playlist đều đã thu thập đủ.

Nhưng nếu thế thì làm sao chúng ta có thể lưu được ID của các tracks nếu thu thập chúng sau khi thu thập playlist? Câu trả lời sẽ có ở phần dưới

In [73]:
def get_track_in_playlist(playlist_link, playlist_name, df_track, trackIds, driver, thread):
    driver.get(playlist_link)
    time.sleep(1)
    track_found = driver.find_elements(By.XPATH, "//a[@class = 'trackItem__trackTitle sc-link-dark sc-link-primary sc-font-light']")
    list_track = []

    for each_res in track_found:
        if each_res.get_attribute('href')!= None and "sets/" in each_res.get_attribute('href'):
            list_track.append(each_res.get_attribute('href'))
    
    for each_track in list_track:
        track = crawl_track_data(each_track, driver)
        track.append(each_track)
        track_ID = str(thread) + '_' + str(len(df_track.index)+1)
        track.insert(1, playlist_name)
        track.insert(0, track_ID)
        df_track.loc[len(df_track)] = track
        trackIds += track_ID + ","
    trackIds = trackIds.rstrip(trackIds[-1]) 
    return trackIds  
    

In [94]:

def crawl_user(gallery_list, thread, df_user, df_playlist):
    driver_ =  webdriver.Chrome(service=s, options = chrome_options)
    #Step 2: With each song in playlist
    user_link = []
    for each_gallery in gallery_list:
        print(each_gallery)
        if 'discover/' not in each_gallery:
            user_link.append(each_gallery[:each_gallery.rfind("/sets")])
        user_link.extend(get_user_link(each_gallery, driver_))
    df_user_link = pd.DataFrame(columns = ['user_link'])

    df_user_link = df_user_link.append(pd.DataFrame({'user_link' : user_link}))
    df_user_link = df_user_link.drop_duplicates()
    user_link = df_user_link['user_link'].tolist()
    each_user = ""
    index = 0
    while user_link != []:
        try:
            if len(df_user.index) > 1000:
                return
            #Step 3: Get link to playlists of user (if exist)
            each_user = user_link.pop(index)
            if each_user in df_user.Link:
                continue
            print(each_user)
            user_info = crawl_data_from_user_link(each_user, driver_)   #user_info[0] : user data; user_info[1] : link playlists
            #Save user data to df_user
            user_info[0].append(each_user)
            user_ID = str(thread) + '_' + str(len(df_user.index)+1)
            user_info[0].insert(0, user_ID)
            df_user.loc[len(df_user)] = user_info[0]
            if user_info[1] != 'None':
                for each_playlist in user_info[1]:
                    playlist_info = getInfoPlaylist(each_playlist, df_track, driver_)
                    playlist_info.append(each_playlist)
                    playlist_ID = str(thread) + '_' + str(len(df_playlist.index)+1)
                    playlist_info.insert(0, playlist_ID)
                    df_playlist.loc[len(df_playlist)] = playlist_info
        except:        
            pass



In [77]:
def crawl_track(df_playlist, df_track, thread, index_playlist):
    track_driver = webdriver.Chrome(service=s, options = chrome_options)
    link_list = df_playlist["Link"].tolist()
    i = 0
    while len(df_track.index) < 1001:
        trackIds = ""
        trackIds = get_track_in_playlist(link_list[i+index_playlist], df_playlist.loc[i + index_playlist, "Playlist name"], df_track, trackIds, track_driver, thread)
        df_playlist.loc[i + index_playlist, "Track IDs"] = trackIds
        print(df_playlist.loc[i + index_playlist, "Playlist name"])
        df_playlist.to_csv("Crawl_data/playlist.csv", sep = '\t', index = False)
        df_track.to_csv("Crawl_data/track.csv", sep = '\t', index = False)
        i+=1


***
## Chia luồng và thực hiện thu thập dữ liệu
### Đặt vấn đề
Trong quá trình thực hiện thu thập dữ liệu bằng cách parse HTML, nhóm đã gặp một ít khó khăn về mặt thời gian. 

Ban đầu nhóm tiến hành chạy và không chia luồng, việc thu thập diễn ra rất lâu do phải truy xuất từ trang chủ vào đến từng bài hát. Thêm nữa, cách code của phần này rất không "clean" và bị lồng vòng lặp nhiều. Kết quả test khi đó rất tệ, cụ thể:
- Test lấy 20 users : chạy trong 90', lấy được khoảng hơn 150 playlists, track đạt yêu cầu
- Test lấy 10 users : chạy trong 30', lấy được khoảng 30 playlists
- Test không giới hạn: với hơn 800' khi treo máy từ ban đêm, số user lấy được chỉ là hơn 60
Do vậy, nếu không thực hiện chia luồng để chạy thì sẽ mất khoảng vài ngày chỉ để thu thập được 1000 record ở mỗi file csv :))))

Ngoài ra, việc chia luồng cũng đảm bảo cho quá trình chạy ít bị gián đoạn, ở lần test gần nhất, lúc parse HTML với track đã có 2 threads bị lỗi, nhưng các threads còn lại vẫn chạy ổn định và cho kết quả trong thời gian dự tính.
### Thu thập dữ liệu từ discover
Việc thu thập dữ liệu bắt đầu từ trang chủ, các mục ở đây cũng được xem như là playlist(trừ các gallery do soundcloud tạo như Top 50...)

Có khá nhiều mục để thu thập, tuy nhiên với 40 mục có thể lấy được (không cần dùng hàm nhấn các button slide bar để load các mục bị ẩn), số lượng user đã đủ để thỏa yêu cầu đề bài (khoảng hơn 1400)

Sau khi thu thập xong các link dẫn tới từng mục, thành các phần nhỏ tương ứng và thực hiện chia luồng chạy

### Đối với user và playlist
Sau khi lấy được link dẫn tới từng user, tại đây thực hiện lấy luôn dữ liệu của từng playlist mà user đó có

Lý do đẻ thực hiện gộp chung 2 việc này lại:
- Số lượng playlist trong mỗi user không quá nhiều, kết quả cho thấy só playlist trung bình của mỗi user thường là 4-6
- Các user ở đây đa phần là những người làm nhạc/sưu tầm nhạc, nên các playlist ở đây cũng sẽ dễ đảm bảo được đủ dữ liệu và ít bị trùng lặp

### Đối với track
Track được tiến hành lấy riêng sau khi đã có dữ liệu của playlist (đã đề cập ở trên)

Các playlist sau khi lấy sẽ phải để mục TrackIDs là None. Tuy có hơi mất một ít thời gian khi phải đặt lại giá trị cho từng playlist, tuy nhiên nó không đáng kể so với thời gian nếu thực hiện lấy playlist và track chung với nhau. Ngoài ra Data Frame của thư viện pandas cũng hỗ trợ công việc này rất tốt thông qua hàm DataFrame.loc()

Để đảm bảo thời gian chạy không quá lâu mà vẫn đủ yêu cầu đề bài thì ở đây chỉ chạy đến khi đủ 1000 record, do vậy sẽ có khá nhiều playlist không được điền TrackIDs (không phải là không thể, chỉ là được giới hạn lại)

In [ ]:
#Step 1: get link from discover to each track gallery
gallery_list = get_all_gallery_in_home(driver)
chunks = [gallery_list[x:x+4] for x in range(0, len(gallery_list), 4)]
number_of_threads = 10
df_user = pd.DataFrame(columns= ['ID', 'User name', 'Name', 'Location', 'Tracks', 'Playlists', 'Followers', 'Following', 'Link'])
df_playlist = pd.DataFrame(columns=['ID', 'Playlist name', 'Playlist owner', 'Release date', 'Number tracks', 'Total time', 'Track IDs', 'Like', 'Repost', 'Link'])
df_track = pd.DataFrame(columns=['ID', 'Name', 'In playlist', 'Owner', 'Release date', 'Duration', 'Play', 'Like', 'Repost', 'Number comment', 'Link']) 
threads_1 = []
print(len(gallery_list))
for thread in range(number_of_threads):
    t = Thread(
        target=crawl_user, 
        args=(chunks[thread], thread, df_user, df_playlist,)) 
    t.daemon = True
    t.start()
    threads_1.append(t)

for t in threads_1:
    t.join()
df_user.to_csv("Crawl_data/user.csv", sep = '\t', index = False)

In [81]:
df_user

,ID,User name,Name,Location,Tracks,Playlists,Followers,Following,Link
0,1_1,Mee Music,Mee Media,"Thai Binh, Viet Nam",14,0,"5,990",230,https://soundcloud.com/meemedia
1,0_2,Hustlang Winno 🐉🐍,Hoàng Long Nguyễn,"Da Nang, Viet Nam",25,0,10.2K,241,https://soundcloud.com/winnodamidside
2,7_3,Billie Kihega,Billie Kihega,None,0,4,"1,427",11,https://soundcloud.com/billie-kihega
3,0_4,Qi,None,Viet Nam,9,2,141,5,https://soundcloud.com/user-73277016
4,6_5,Mah,None,None,25,9,"5,151",142,https://soundcloud.com/mzn665
...,...,...,...,...,...,...,...,...,...
996,5_997,hypnonys,Min Kang,"Seoul, Republic of Korea",14,0,188,0,https://soundcloud.com/hypnonys
997,5_998,Newaile(뉴엘르),None,None,21,0,"1,386",26,https://soundcloud.com/newaile
998,2_999,Big Beat Records Verified,None,New York,918,28,751K,265,https://soundcloud.com/wearebigbeat
999,5_1000,chobizzy Verified,Soohyung Cho,"Tokyo, Japan",135,18,"4,157",948,https://soundcloud.com/soohyungcho


In [68]:
df_playlist

,ID,Playlist name,Playlist owner,Release date,Number tracks,Total time,Track IDs,Like,Repost,Link
0,7_1,Billies list,Billie Kihega,17 September 2019,2,6:10,None,356,14,https://soundcloud.com/billie-kihega/sets/bill...
1,0_2,Chill 30/4,Qi,30 April 2021,27,1:32:32,None,1,0,https://soundcloud.com/user-73277016/sets/chil...
2,7_3,Billies Sleep Sounds,Billie Kihega,16 February 2017,13,4:46:27,None,"41,123","2,481",https://soundcloud.com/billie-kihega/sets/bill...
3,6_4,گرامافون,Mah,11 November 2019,18,2:09:54,None,123,3,https://soundcloud.com/mzn665/sets/gramophone
4,1_5,Bác sĩ Hải's Deep mix,Bác sĩ Hải,25 September 2021,19,1:19:55,None,230,23,https://soundcloud.com/bacsihai/sets/b-c-s-h-i...
...,...,...,...,...,...,...,...,...,...,...
5595,8_5596,Jerro feat. Kauf - Tunnel Vision (Original and...,Anjunadeep,20 April 2020,2,9:22,None,103,38,https://soundcloud.com/anjunadeep/sets/jerro-f...
5596,8_5597,Luttrell - Lucky Ones (The Remixes: Part 1),Anjunadeep,17 April 2020,2,11:44,None,68,18,https://soundcloud.com/anjunadeep/sets/luttrel...
5597,8_5598,Nikola Melnikov & Max Sorokin - Sun Wakes Up E...,Anjunadeep,27 March 2020,2,13:16,None,159,45,https://soundcloud.com/anjunadeep/sets/nikola-...
5598,8_5599,Moon Boots feat. Nic Hanson - Clear (The Remixes),Anjunadeep,26 March 2020,2,11:21,None,65,17,https://soundcloud.com/anjunadeep/sets/moon-bo...


In [ ]:
threads_2 = []
for thread in range(10):
    t = Thread(
        target=crawl_track, 
        args=(df_playlist, df_track, thread, thread*100,)) 
    t.daemon = True
    t.start()
    threads_2.append(t)

for t in threads_2:
    t.join()


In [82]:
df_track

,ID,Name,In playlist,Owner,Release date,Duration,Play,Like,Repost,Number comment,Link
0,3_1,Tiësto - Color Switch Phoenix (Color Switch So...,Color Switch Phoenix,Musical Freedom,21 August 2018,2:21,142K,"3,128",434,58,https://soundcloud.com/musical-freedom/tiesto-...
1,0_2,bad guy,Billies list,Billie Eilish,29 March 2019,0:30,45.3M,741K,18K,0,https://soundcloud.com/billieeilish/bad-guy?in...
2,7_3,Chino Espinoza - Yeah' (Como Goza Mi Morena) (...,chacha,Wuchang,6 January 2012,3:57,"6,878",58,3,0,https://soundcloud.com/wuchang/chino-espinoza-...
3,2_4,"Bác sĩ Hải , GiangTo - Chưa Bao giờ mẹ Kể rem...","Bác sĩ Hải , GiangTo - Chưa Bao giờ mẹ Kể rem...",Bác sĩ Hải,16 October 2021,3:44,48.8K,"1,039",76,27,https://soundcloud.com/bacsihai/min-erik-chua-...
4,6_5,Spliff,#1 MS DOCKVILLE 2016,Bilderbuch,29 October 2014,3:01,209K,"4,048",414,32,https://soundcloud.com/bilderbuch/spliff?in=do...
...,...,...,...,...,...,...,...,...,...,...,...
1054,3_1055,J. Cole - Welcome,KOD - J. Cole Greatest Hits,KBGetem,31 October 2011,2:20,752K,10.6K,680,56,https://soundcloud.com/kbgetem/j-cole-welcome?...
1055,3_1056,12-J Cole-The Autograph,KOD - J. Cole Greatest Hits,highleague,14 November 2010,3:43,262K,"3,373",304,11,https://soundcloud.com/highleague/12-j-cole-th...
1056,3_1057,15 - Premeditated Murder,KOD - J. Cole Greatest Hits,newjcole,19 June 2011,3:54,997K,11.1K,889,21,https://soundcloud.com/newjcole/15-premeditate...
1057,2_1058,XO,feels,EDEN,13 April 2015,2:39,16.8M,225K,19.4K,"2,233",https://soundcloud.com/iameden/x-o?in=awk-v/se...


***
# Tổng kết
***

Việc thực hiện lấy dữ liệu bằng HTML là mất thời gian hơn bằng API tương đối nhiều, tuy nhiên với cách code khác đi đã cho ra kết quả tương đối tốt:
- Đôi với user và playlist (chưa update TrackIDs): 160' với 1000 users và 5600 playlists
- Đối với track (sau khi điền luôn cả một ít TrackIDS): 30' cho 1000 records
- Số trường của user.csv: 9
- Số trường của playlist.csv: 10
- Số trường của track.csv: 11

Kết quả bài làm sẽ được nộp kèm theo, code được viết để lưu dữ liệu sau khi chạy xong nên trong quá trình chạy thầy có thể kiểm tra các file dữ liệu.

## Nguồn tham khảo
https://stackoverflow.com/questions/41354472/multi-threading-in-selenium-python

https://stackoverflow.com/questions/59387309/multithreading-in-python-selenium